# XGboost Project

In the two previous projects we saw how we could use a decision tree and then a random forest to improve the prediction of diabetes. We have reached a point where we need to improve. Can boosting be the best alternative to optimize the results?

Boosting is a sequential composition of models (usually decision trees) in which the new model aims to correct the errors of the previous one. This view may be useful in this data set, since several of the assumptions studied in the module are met.

In [18]:
import pandas as pd
import optuna


In [8]:
# Loading data of Diabetes prediction dataset with and without outliers

outliers_data = pd.read_excel("/workspaces/xgboost-project/data/processed/outliers_data.xlsx")
no_outliers_data = pd.read_excel("/workspaces/xgboost-project/data/processed/no_outliers_data.xlsx")

outliers_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
no_outliers_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,2.959184,122.627551,70.596939,29.066327,130.660714,32.697704,0.491561,29.589286,0.331633
std,2.744605,30.860781,11.306056,10.376823,71.308664,6.310185,0.264094,8.639242,0.471401
min,0.000000,56.000000,38.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,113.000000,32.900000,0.449500,26.000000,0.000000
75%,4.000000,143.000000,78.000000,36.250000,172.000000,36.800000,0.687000,34.000000,1.000000
max,11.000000,198.000000,102.000000,60.000000,342.000000,49.700000,1.292000,55.000000,1.000000


In [12]:
# split both datasets into train and test

from sklearn.model_selection import train_test_split

#data with outliers 
predictors = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
target = 'Outcome'

X = outliers_data[predictors]
y = outliers_data[target]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


#data without outliers

predictors1 = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
target1 = 'Outcome'

X = no_outliers_data[predictors1]
y = no_outliers_data[target1]

#data with outliers no scaled
X_train_no_out, X_test_no_out, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
Mejores parámetros:  {'n_estimators': 218, 'learning_rate': 0.0899598553283333, 'subsample': 0.6126950362885215, 'max_depth': 6, 'gamma': 0.0012937020191838994, 'min_child_weight': 9.179599573247735, 'colsample_bylevel': 0.6714388804457142}


### Hyperparameter Rendering (Optuna)

In [28]:
import xgboost as xgb


def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.3),
        'objective': 'binary:logistic',
        'subsample': trial.suggest_loguniform('subsample', 0.5, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-8, 10.0),
        'colsample_bylevel': trial.suggest_loguniform('colsample_bylevel', 0.5, 1.0),
        'random_state': 42
    }
    
    # Crear y entrenar el modelo
    model = xgb.XGBClassifier(**param)
    model.fit(X_train, y_train)
    
    # Predecir y calcular la precisión
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

In [29]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print('Número de pruebas: ', len(study.trials))
print('Mejores parámetros: ', study.best_params)
print('Mejor precisión: ', study.best_value)

[I 2024-05-15 18:47:13,671] A new study created in memory with name: no-name-a155fa8f-6a14-421f-80f0-a3b89e32e203
/tmp/ipykernel_646/892496538.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.3),
/tmp/ipykernel_646/892496538.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'subsample': trial.suggest_loguniform('subsample', 0.5, 1.0),
/tmp/ipykernel_646/892496538.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_lo

Número de pruebas:  100
Mejores parámetros:  {'n_estimators': 289, 'learning_rate': 0.14197104731466412, 'subsample': 0.5819998851844587, 'max_depth': 6, 'gamma': 7.689364666271964e-06, 'min_child_weight': 0.00012105325271544954, 'colsample_bylevel': 0.7425979993167775}
Mejor precisión:  0.7848101265822784


#### XGboost Model with outliers

In [33]:
from xgboost import XGBClassifier

model_outliers = XGBClassifier(n_estimators = 200, learning_rate = 0.0899598553283333, subsample = 0.6126950362885215, max_depth = 6, gamma = 0.0012937020191838994, min_child_weight = 9.179599573247735, colsample_bylevel = 0.6714388804457142, random_state = 42)
model_outliers.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.6714388804457142, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0012937020191838994, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0899598553283333, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=9.179599573247735, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [34]:
# prediction

y_pred = model_outliers.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [35]:
# accuracy model

from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.810126582278481

In [37]:
# save the model

from pickle import dump

dump(model_outliers, open("/workspaces/xgboost-project/models/XGboost_rseed_42.sav", "wb"))